Pronostico de series de tiempo --- 0:00 min
===

* Última modificación: Marzo 7, 2022 | YouTube

Importación de librerías
---

In [12]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

tf.__version__

'2.8.0'

In [ ]:
mpl.rcParams["figure.figsize"] = (8, 6)
mpl.rcParams["axes.grid"] = False

Dataset
--

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip",
    fname="jena_climate_2009_2016.csv.zip",
    extract=True,
)
csv_path, _ = os.path.splitext(zip_path)

In [ ]:
df = pd.read_csv(csv_path)
df = df[5::6]

date_time = pd.to_datetime(df.pop("Date Time"), format="%d.%m.%Y %H:%M:%S")

In [ ]:
df.head()

In [ ]:
plot_cols = ["T (degC)", "p (mbar)", "rho (g/m**3)"]
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

**Inspección y limpieza**

In [ ]:
df.describe().transpose()

* **Velocidad del viento**

In [ ]:
#
# Velocidad del viento
#
wv = df["wv (m/s)"]
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

max_wv = df["max. wv (m/s)"]
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

df["wv (m/s)"].min()

**Ingeniería de características**

* **Viento**

In [ ]:
plt.hist2d(df["wd (deg)"], df["wv (m/s)"], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel("Wind Direction [deg]")
plt.ylabel("Wind Velocity [m/s]")

In [ ]:
wv = df.pop("wv (m/s)")
max_wv = df.pop("max. wv (m/s)")

wd_rad = df.pop("wd (deg)") * np.pi / 180

df["Wx"] = wv * np.cos(wd_rad)
df["Wy"] = wv * np.sin(wd_rad)

df["max Wx"] = max_wv * np.cos(wd_rad)
df["max Wy"] = max_wv * np.sin(wd_rad)

In [ ]:
plt.hist2d(df["Wx"], df["Wy"], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel("Wind X [m/s]")
plt.ylabel("Wind Y [m/s]")
ax = plt.gca()
ax.axis("tight")

* **Time**

In [ ]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)

In [ ]:
day = 24 * 60 * 60
year = (365.2425) * day

df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
plt.plot(np.array(df["Day sin"])[:25])
plt.plot(np.array(df["Day cos"])[:25])
plt.xlabel("Time [h]")
plt.title("Time of day signal")

In [ ]:
fft = tf.signal.rfft(df["T (degC)"])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(df["T (degC)"])
hours_per_year = 24 * 365.2524
years_per_dataset = n_samples_h / (hours_per_year)

f_per_year = f_per_dataset / years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale("log")
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=["1/Year", "1/day"])
_ = plt.xlabel("Frequency (log scale)")

**Particionamiento de los datos**

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0 : int(n * 0.7)]
val_df = df[int(n * 0.7) : int(n * 0.9)]
test_df = df[int(n * 0.9) :]

num_features = df.shape[1]

**Normalización de los datos**

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

Now, peek at the distribution of the features. Some features do have long tails, but there are no obvious errors like the `-9999` wind velocity value.

In [ ]:
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name="Column", value_name="Normalized")
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x="Column", y="Normalized", data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

Data windowing
--

* Predicción simpre 24 horas en el futuro, usando 24 horas de historia.

![assets/raw_window_24h.png](assets/raw_window_24h.png)

* Predicción de una hora en el futuro usando 6 horas de historia

![assets/raw_window_1h.png](assets/raw_window_1h.png)

**1. Indces y offsets**

In [ ]:
class WindowGenerator:
    def __init__(
        self,
        input_width,
        label_width,
        shift,
        train_df=train_df,
        val_df=val_df,
        test_df=test_df,
        label_columns=None,
    ):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {
                name: i for i, name in enumerate(label_columns)
            }
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return "\n".join(
            [
                f"Total window size: {self.total_window_size}",
                f"Input indices: {self.input_indices}",
                f"Label indices: {self.label_indices}",
                f"Label column name(s): {self.label_columns}",
            ]
        )

In [ ]:
w1 = WindowGenerator(
    input_width=24,
    label_width=1,
    shift=24,
    label_columns=["T (degC)"],
)
w1

In [ ]:
w2 = WindowGenerator(
    input_width=6,
    label_width=1,
    shift=1,
    label_columns=["T (degC)"],
)
w2

**2. Split**

![assets/split_window.png](assets/split_window.png)

In [ ]:
def split_window(self, features):

    inputs = features[:, self.input_slice, :]

    labels = features[:, self.labels_slice, :]

    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1,
        )

    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels


WindowGenerator.split_window = split_window

In [ ]:
example_window = tf.stack(
    [
        np.array(train_df[: w2.total_window_size]),
        np.array(train_df[100 : 100 + w2.total_window_size]),
        np.array(train_df[200 : 200 + w2.total_window_size]),
    ]
)

example_inputs, example_labels = w2.split_window(example_window)

print("All shapes are: (batch, time, features)")
print(f"Window shape: {example_window.shape}")
print(f"Inputs shape: {example_inputs.shape}")
print(f"Labels shape: {example_labels.shape}")

**3. Plot**

In [ ]:
w2.example = example_inputs, example_labels

In [ ]:
def plot(
    self,
    model=None,
    plot_col="T (degC)",
    max_subplots=3,
):

    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        plt.subplot(max_n, 1, n + 1)
        plt.ylabel(f"{plot_col} [normed]")
        plt.plot(
            self.input_indices,
            inputs[n, :, plot_col_index],
            label="Inputs",
            marker=".",
            zorder=-10,
        )

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(
            self.label_indices,
            labels[n, :, label_col_index],
            edgecolors="k",
            label="Labels",
            c="#2ca02c",
            s=64,
        )
        if model is not None:
            predictions = model(inputs)
            plt.scatter(
                self.label_indices,
                predictions[n, :, label_col_index],
                marker="X",
                edgecolors="k",
                label="Predictions",
                c="#ff7f0e",
                s=64,
            )

        if n == 0:
            plt.legend()

    plt.xlabel("Time [h]")


WindowGenerator.plot = plot

In [ ]:
w2.plot()

In [ ]:
w2.plot(plot_col="p (mbar)")

**4. Creación de tf.Dataset**

In [ ]:
def make_dataset(self, data):

    data = np.array(data, dtype=np.float32)

    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,
    )

    ds = ds.map(self.split_window)

    return ds


WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)


@property
def val(self):
    return self.make_dataset(self.val_df)


@property
def test(self):
    return self.make_dataset(self.test_df)


@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, "_example", None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.train))
        # And cache it for next time
        self._example = result
    return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
w2.train.element_spec

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

Modelos de un paso adelante
--

![assets/narrow_window.png](assets/narrow_window.png)

In [ ]:
single_step_window = WindowGenerator(
    input_width=1,
    label_width=1,
    shift=1,
    label_columns=["T (degC)"],
)
single_step_window

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

**Baseline**

![assets/baseline.png](assets/baseline.png)

In [ ]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [ ]:
baseline = Baseline(
    label_index=column_indices["T (degC)"],
)

baseline.compile(
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
    ],
)

val_performance = {}

performance = {}

val_performance["Baseline"] = baseline.evaluate(single_step_window.val)

performance["Baseline"] = baseline.evaluate(
    single_step_window.test,
    verbose=0,
)

In [ ]:
wide_window = WindowGenerator(
    input_width=24,
    label_width=24,
    shift=1,
    label_columns=["T (degC)"],
)

wide_window

![assets/last_window.png](assets/last_window.png)

In [ ]:
print("Input shape:", wide_window.example[0].shape)
print("Output shape:", baseline(wide_window.example[0]).shape)

In [ ]:
wide_window.plot(baseline)

**Modelo lineal**

![assets/narrow_window.png](assets/narrow_window.png)

In [1]:
linear = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units=1),
    ]
)

NameError: name 'tf' is not defined

In [ ]:
print(
    "Input shape:",
    single_step_window.example[0].shape,
)

print(
    "Output shape:",
    linear(single_step_window.example[0]).shape,
)

In [ ]:
MAX_EPOCHS = 20


def compile_and_fit(model, window, patience=2):

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=patience,
        mode="min",
    )

    model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError()],
    )

    history = model.fit(
        window.train,
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping],
    )
    return history

In [ ]:
history = compile_and_fit(
    linear,
    single_step_window,
)

val_performance["Linear"] = linear.evaluate(
    single_step_window.val,
)

performance["Linear"] = linear.evaluate(
    single_step_window.test,
    verbose=0,
)

![assets/wide_window.png](assets/wide_window.png)

In [ ]:
print("Input shape:", wide_window.example[0].shape)
print("Output shape:", baseline(wide_window.example[0]).shape)

In [ ]:
wide_window.plot(linear)

In [ ]:
plt.bar(x=range(len(train_df.columns)), height=linear.layers[0].kernel[:, 0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

Sometimes the model doesn't even place the most weight on the input `T (degC)`. This is one of the risks of random initialization. 

**Modelo Denso**

In [ ]:
dense = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            units=64,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=64,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=1,
        ),
    ]
)

history = compile_and_fit(
    dense,
    single_step_window,
)

val_performance["Dense"] = dense.evaluate(
    single_step_window.val,
)

performance["Dense"] = dense.evaluate(
    single_step_window.test,
    verbose=0,
)

**Modelo denso multipaso**

![assets/conv_window.png](assets/conv_window.png)

In [ ]:
CONV_WIDTH = 3

conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=[
        "T (degC)",
    ],
)

conv_window

In [ ]:
conv_window.plot()
plt.title("Given 3 hours of inputs, predict 1 hour into the future.")

In [ ]:
multi_step_dense = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(
            units=32,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=32,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=1,
        ),
        tf.keras.layers.Reshape([1, -1]),
    ]
)

In [ ]:
print("Input shape:", conv_window.example[0].shape)
print("Output shape:", multi_step_dense(conv_window.example[0]).shape)

In [ ]:
from IPython.display import clear_output

history = compile_and_fit(
    multi_step_dense,
    conv_window,
)

clear_output()

val_performance["Multi step dense"] = multi_step_dense.evaluate(
    conv_window.val,
)

performance["Multi step dense"] = multi_step_dense.evaluate(
    conv_window.test,
    verbose=0,
)

In [ ]:
conv_window.plot(multi_step_dense)

In [ ]:
print("Input shape:", wide_window.example[0].shape)
try:
    print("Output shape:", multi_step_dense(wide_window.example[0]).shape)
except Exception as e:
    print(f"\n{type(e).__name__}:{e}")

**Red neuronal convolucional**

In [ ]:
conv_model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv1D(
            filters=32,
            kernel_size=(CONV_WIDTH,),
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=32,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            units=1,
        ),
    ]
)

In [ ]:
print("Conv model on `conv_window`")
print("Input shape:", conv_window.example[0].shape)
print("Output shape:", conv_model(conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance["Conv"] = conv_model.evaluate(conv_window.val)
performance["Conv"] = conv_model.evaluate(conv_window.test, verbose=0)

![assets/wide_conv_window.png](assets/wide_conv_window.png)

In [ ]:
print("Wide window")
print("Input shape:", wide_window.example[0].shape)
print("Labels shape:", wide_window.example[1].shape)
print("Output shape:", conv_model(wide_window.example[0]).shape)

In [ ]:
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)

wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    label_columns=["T (degC)"],
)

wide_conv_window

In [ ]:
print("Wide conv window")
print("Input shape:", wide_conv_window.example[0].shape)
print("Labels shape:", wide_conv_window.example[1].shape)
print("Output shape:", conv_model(wide_conv_window.example[0]).shape)

In [ ]:
wide_conv_window.plot(conv_model)

**Red neuronal recurrente LSTM**

```python
return_sequences==False
````

![assets/lstm_1_window.png](assets/lstm_1_window.png)

```python
return_sequences==True
````

![asets/lstm_many_window.png](assets/lstm_many_window.png)

In [ ]:
lstm_model = tf.keras.models.Sequential(
    [
        # Shape [batch, time, features] => [batch, time, lstm_units]
        tf.keras.layers.LSTM(32, return_sequences=True),
        # Shape => [batch, time, features]
        tf.keras.layers.Dense(units=1),
    ]
)

In [ ]:
print("Input shape:", wide_window.example[0].shape)
print("Output shape:", lstm_model(wide_window.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, wide_window)

clear_output()

val_performance["LSTM"] = lstm_model.evaluate(wide_window.val)

performance["LSTM"] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

**Desempeño**

In [ ]:
x = np.arange(len(performance))
width = 0.3
metric_name = "mean_absolute_error"
metric_index = lstm_model.metrics_names.index("mean_absolute_error")
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.ylabel("mean_absolute_error [T (degC), normalized]")
plt.bar(x - 0.17, val_mae, width, label="Validation")
plt.bar(x + 0.17, test_mae, width, label="Test")
plt.xticks(ticks=x, labels=performance.keys(), rotation=45)
_ = plt.legend()

In [ ]:
for name, value in performance.items():
    print(f"{name:12s}: {value[1]:0.4f}")

Modelos de múltiples salidas
--

In [ ]:
single_step_window = WindowGenerator(
    input_width=1,
    label_width=1,
    shift=1,
)

wide_window = WindowGenerator(
    input_width=24,
    label_width=24,
    shift=1,
)

for example_inputs, example_labels in wide_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

Note above that the `features` axis of the labels now has the same depth as the inputs, instead of `1`.

**Modelo base**

In [ ]:
baseline = Baseline()
baseline.compile(
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
    ],
)

In [ ]:
val_performance = {}
performance = {}
val_performance["Baseline"] = baseline.evaluate(wide_window.val)
performance["Baseline"] = baseline.evaluate(wide_window.test, verbose=0)

**Modelo denso**

In [ ]:
dense = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dense(units=num_features),
    ]
)

In [ ]:
history = compile_and_fit(dense, single_step_window)

IPython.display.clear_output()
val_performance["Dense"] = dense.evaluate(single_step_window.val)
performance["Dense"] = dense.evaluate(single_step_window.test, verbose=0)

**RNN**


In [ ]:
%%time
wide_window = WindowGenerator(input_width=24, label_width=24, shift=1)

lstm_model = tf.keras.models.Sequential(
    [
        # Shape [batch, time, features] => [batch, time, lstm_units]
        tf.keras.layers.LSTM(32, return_sequences=True),
        # Shape => [batch, time, features]
        tf.keras.layers.Dense(units=num_features),
    ]
)

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance["LSTM"] = lstm_model.evaluate(wide_window.val)
performance["LSTM"] = lstm_model.evaluate(wide_window.test, verbose=0)

print()

**Conexiones de residuales**

![assets/residual.png](assets/residual.png)

In [ ]:
class ResidualWrapper(tf.keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def call(self, inputs, *args, **kwargs):
        delta = self.model(inputs, *args, **kwargs)
        return inputs + delta

In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential(
        [
            tf.keras.layers.LSTM(
                32,
                return_sequences=True,
            ),
            tf.keras.layers.Dense(
                num_features,
                kernel_initializer=tf.initializers.zeros(),
            ),
        ]
    )
)

history = compile_and_fit(residual_lstm, wide_window)

clear_output()
val_performance["Residual LSTM"] = residual_lstm.evaluate(
    wide_window.val,
)
performance["Residual LSTM"] = residual_lstm.evaluate(
    wide_window.test,
    verbose=0,
)
print()

**Desempeño**

In [ ]:
x = np.arange(len(performance))
width = 0.3

metric_name = "mean_absolute_error"
metric_index = lstm_model.metrics_names.index("mean_absolute_error")
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label="Validation")
plt.bar(x + 0.17, test_mae, width, label="Test")
plt.xticks(ticks=x, labels=performance.keys(), rotation=45)
plt.ylabel("MAE (average over all outputs)")
_ = plt.legend()

In [ ]:
for name, value in performance.items():
    print(f"{name:15s}: {value[1]:0.4f}")

Modelos multipaso
--

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24, label_width=OUT_STEPS, shift=OUT_STEPS)

multi_window.plot()
multi_window

**Modelos base**

![assets/multistep_last.png](assets/multistep_last.png)

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
    def call(self, inputs):
        return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])


last_baseline = MultiStepLastBaseline()
last_baseline.compile(
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
    ],
)

multi_val_performance = {}
multi_performance = {}

multi_val_performance["Last"] = (
    last_baseline.evaluate(
        multi_window.val,
    ),
)
multi_performance["Last"] = last_baseline.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(last_baseline)

![assets/multistep_repeat.png](assets/multistep_repeat.png)

In [ ]:
class RepeatBaseline(tf.keras.Model):
    def call(self, inputs):
        return inputs


repeat_baseline = RepeatBaseline()
repeat_baseline.compile(
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
    ],
)

multi_val_performance["Repeat"] = repeat_baseline.evaluate(
    multi_window.val,
)
multi_performance["Repeat"] = repeat_baseline.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(repeat_baseline)

**Single-shot models**

* **Linear**

![assets/multistep_dense.png](assets/multistep_dense.png)

In [ ]:
multi_linear_model = tf.keras.Sequential(
    [
        tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
        tf.keras.layers.Dense(
            OUT_STEPS * num_features,
            kernel_initializer=tf.initializers.zeros(),
        ),
        tf.keras.layers.Reshape([OUT_STEPS, num_features]),
    ]
)

history = compile_and_fit(multi_linear_model, multi_window)

IPython.display.clear_output()
multi_val_performance["Linear"] = multi_linear_model.evaluate(
    multi_window.val,
)
multi_performance["Linear"] = multi_linear_model.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(
    multi_linear_model,
)

* **Dense**

In [ ]:
multi_dense_model = tf.keras.Sequential(
    [
        tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
        tf.keras.layers.Dense(
            512,
            activation="relu",
        ),
        tf.keras.layers.Dense(
            OUT_STEPS * num_features,
            kernel_initializer=tf.initializers.zeros(),
        ),
        tf.keras.layers.Reshape(
            [
                OUT_STEPS,
                num_features,
            ]
        ),
    ]
)

history = compile_and_fit(
    multi_dense_model,
    multi_window,
)

clear_output()
multi_val_performance["Dense"] = multi_dense_model.evaluate(
    multi_window.val,
)
multi_performance["Dense"] = multi_dense_model.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(
    multi_dense_model,
)

* **CNN**

![assets/multistep_conv.png](assets/multistep_conv.png)

In [ ]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential(
    [
        tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
        tf.keras.layers.Conv1D(
            256,
            activation="relu",
            kernel_size=(CONV_WIDTH),
        ),
        tf.keras.layers.Dense(
            OUT_STEPS * num_features,
            kernel_initializer=tf.initializers.zeros(),
        ),
        tf.keras.layers.Reshape(
            [
                OUT_STEPS,
                num_features,
            ]
        ),
    ]
)

history = compile_and_fit(
    multi_conv_model,
    multi_window,
)

clear_output()

multi_val_performance["Conv"] = multi_conv_model.evaluate(
    multi_window.val,
)
multi_performance["Conv"] = multi_conv_model.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(
    multi_conv_model,
)

* **RNN**

![assets/multistep_lstm.png](assets/multistep_lstm.png)

In [ ]:
multi_lstm_model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(
            32,
            return_sequences=False,
        ),
        tf.keras.layers.Dense(
            OUT_STEPS * num_features,
            kernel_initializer=tf.initializers.zeros(),
        ),
        tf.keras.layers.Reshape(
            [
                OUT_STEPS,
                num_features,
            ]
        ),
    ]
)

history = compile_and_fit(
    multi_lstm_model,
    multi_window,
)

clear_output()

multi_val_performance["LSTM"] = multi_lstm_model.evaluate(
    multi_window.val,
)
multi_performance["LSTM"] = multi_lstm_model.evaluate(
    multi_window.test,
    verbose=0,
)
multi_window.plot(
    multi_lstm_model,
)

**Modelo autoregresivo**

![assets/multistep_autoregressive.png](assets/multistep_autoregressive.png)

* **RNN**

In [ ]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

In [ ]:
def warmup(self, inputs):
    x, *state = self.lstm_rnn(inputs)
    prediction = self.dense(x)
    return prediction, state


FeedBack.warmup = warmup

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

In [ ]:
def call(self, inputs, training=None):

    predictions = []

    prediction, state = self.warmup(inputs)

    predictions.append(prediction)

    for n in range(1, self.out_steps):
        x = prediction
        x, state = self.lstm_cell(
            x,
            states=state,
            training=training,
        )
        prediction = self.dense(x)
        predictions.append(prediction)

    predictions = tf.stack(predictions)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions


FeedBack.call = call

In [ ]:
print(
    "Output shape (batch, time, features): ",
    feedback_model(multi_window.example[0]).shape,
)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

IPython.display.clear_output()

multi_val_performance["AR LSTM"] = feedback_model.evaluate(multi_window.val)
multi_performance["AR LSTM"] = feedback_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(feedback_model)

**Desempeño**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3

metric_name = "mean_absolute_error"
metric_index = lstm_model.metrics_names.index("mean_absolute_error")
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_mae, width, label="Validation")
plt.bar(x + 0.17, test_mae, width, label="Test")
plt.xticks(ticks=x, labels=multi_performance.keys(), rotation=45)
plt.ylabel(f"MAE (average over all times and outputs)")
_ = plt.legend()

In [ ]:
for name, value in multi_performance.items():
    print(f"{name:8s}: {value[1]:0.4f}")